# Descartar los ligandos enterrados

Puedo utilizar el módulo `ShrakeRupley` de `Bio.PDB.SASA` de `Biopython`.

https://biopython.org/docs/dev/api/Bio.PDB.SASA.html

La idea sería:

- Parsear la estructura
- Crear una copia del ligando libre
- Calcular la superficie accesible al solvente de ligando libre
- Calcular la superficie accesible al solvente del ligando unido
- Calcular la fracción de superficie solvente accesible enterrada
- Descartar las entradas con una fracción de superficie accesible al solvente < 0.2.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install biopython

In [ ]:
#@title Abrir el dataframe

import os
import pandas as pd

input_folder = "/content/drive/MyDrive/TMF/T1/LIGANDO_ENTERRADO" # Ruta del df armonizado sin entradas covalentes
input_file = os.path.join(input_folder, "df_sin_covalentes.csv") # Nombre df armonizado sin entradas covalentes
df_harm = pd.read_csv(input_file, sep = ',')
print(df_harm.shape[0],df_harm.columns)

In [ ]:
#@title Calcular Fracción Accesible al solvente

import os
from Bio.PDB.Polypeptide import is_aa
from Bio.PDB import MMCIFParser
from Bio.PDB.SASA import ShrakeRupley
from Bio.PDB import MMCIFParser
import pandas as pd

sr_ligand_free = ShrakeRupley()
sr_ligand_bind = ShrakeRupley()

id_pdb_correct_1 = []
id_pdb_remove_1 = []
entradas_coordenadas = {}

parser = MMCIFParser(QUIET=True)
input_path = "/content/drive/MyDrive/TMF/T1/ENLACE_COVALENTE/PDB-CAT/PDB-CAT/cif/9"

for index,pdb_id_armonizado in enumerate(df_harm['PDB_entry_id']):
  cif_file = os.path.join(input_path, f"{pdb_id_armonizado.lower()}.cif")

  if os.path.exists(cif_file):
    estructura = parser.get_structure(f"estructura_{pdb_id_armonizado.lower()}",cif_file)

  # Encontrar el ligando
    for modelo in estructura:
      for cadena in modelo:
        for residuo in cadena:
          if residuo.resname == df_harm.loc[index,'Ligand_id']:
            ligand = residuo
          #print(f"Entrada PDB: {pdb_id_armonizado}, Nombre ligando: {ligand.resname}")

          # Calcular la sasa del ligando libre
            sr_ligand_free.compute(ligand)
            sasa_ligand_free = sum(atom.sasa for atom in ligand.get_atoms())
          #print("SASA del ligando libre:", sasa_ligand_free)

          # Calcular la sasa del ligando unido
            sr_ligand_bind.compute(estructura)
            sasa_ligand_bind = sum(atom.sasa for atom in ligand.get_atoms())
          #print("SASA del ligando unido:", sasa_ligand_bind)

            fraccion_enterrada = (sasa_ligand_bind/sasa_ligand_free*100)
          #print(f"Fracción enterrada: {fraccion_enterrada.round(2)}%")
            if fraccion_enterrada >= 20.0:
            #print('Correcto')
              id_pdb_correct_1.append(pdb_id_armonizado)

              if pdb_id_armonizado not in entradas_coordenadas:
                entradas_coordenadas[pdb_id_armonizado] = []
                coordenadas = [cadena.id, residuo.id[1]]
                if coordenadas not in entradas_coordenadas[pdb_id_armonizado]:
                  entradas_coordenadas[pdb_id_armonizado].append(coordenadas)

                  print(f"{fraccion_enterrada.round(2)}, {pdb_id_armonizado}: Correcto")
                  print(entradas_coordenadas)

            else:
            #print('Descartar')
              id_pdb_remove_1.append(pdb_id_armonizado)
              print(f"{fraccion_enterrada.round(2)}, {pdb_id_armonizado}: Descartar")
              #print("\n")
df_enterrado_8 = pd.DataFrame(entradas_coordenadas)
output_file = os.path.join(input_folder,"df_enterrado_9.csv")
df_enterrado_8.to_csv(output_file, index=False, sep = ',')

In [ ]:
def recalcular_fraccion_enterrada_proteica(row, path):
    from Bio.PDB.Polypeptide import is_aa
    import os
    from Bio.PDB import MMCIFParser
    from Bio.PDB.SASA import ShrakeRupley
    from Bio.PDB import MMCIFParser
    import pandas as pd
    from pandarallel import pandarallel
    from Bio.PDB import Model

    entrada = row['PDB_entry_id']
    ligand_id = row['Ligand_id']
    coords = row['Coordenadas']

    if coords is None or len(coords) != 4:
        return None

    modelo_id, cadena_id, residuo_num = coords[1], coords[2], coords[3]
    cif_file = os.path.join(path, f"{entrada.lower()}.cif")

    if not os.path.isfile(cif_file):
        return None

    parser = MMCIFParser(QUIET=True)
    estructura = parser.get_structure(f"estructura_{entrada.lower()}", cif_file)

    try:
        modelo = estructura[modelo_id]
        cadena = modelo[cadena_id]
        residuo = cadena[('H_' + ligand_id, residuo_num, ' ')]  # Ligandos son heteroátomos
    except KeyError:
        return None

    sr = ShrakeRupley()

    # Calcular SASA del ligando libre
    sr.compute(residuo)
    sasa_ligand_free = sum(atom.sasa for atom in residuo.get_atoms())
    if sasa_ligand_free == 0:
        return None

    # Crear nuevo modelo solo con proteína y ligando
    nuevo_modelo = Model.Model(0)
    cadenas_agregadas = set()

    for c in modelo:
        if any(is_aa(r, standard=True) for r in c):
            nuevo_modelo.add(c)
            cadenas_agregadas.add(c.id)

    # Agregar la cadena del ligando si NO está ya incluida
    if cadena_id not in cadenas_agregadas:
        nuevo_modelo.add(cadena)

    # Calcular SASA del ligando en entorno proteico
    sr.compute(nuevo_modelo)
    sasa_ligand_bind = sum(atom.sasa for atom in residuo.get_atoms())

    # Fracción enterrada
    fraccion_enterrada = (1 - sasa_ligand_bind / sasa_ligand_free) * 100
    fraccion_enterrada = round(fraccion_enterrada, 2)

    return fraccion_enterrada

In [ ]:
path_to_cif_files = "/content/drive/MyDrive/TFM/T2/CIF_DF"

In [ ]:
from pandarallel import pandarallel

pandarallel.initialize(nb_workers= 2, progress_bar= True)

df_harm['Fraccion_enterrada'] = df_harm.parallel_apply(
        lambda row: recalcular_fraccion_enterrada_proteica_1(row, path_to_cif_files), axis=1)